# Présentation du projet Python M2S1

## Guide pour l'utilisateur

### Phase de scraping avec le module scrap_main.py

1. Xpath et RE pour Decathlon

In [2]:
import scrap_main

Cet import ouvre la fenètre de navigation. Il faut donc restart le kernel avant de scraper un autre site.

In [3]:
elemedecat = scrap_main.PageElem(
    nom_site= "Decathlon",
    adresse = "https://www.decathlontravel.com/voyages/sport-rando-nature",
    bouton_accept = '//*[@id="didomi-notice-agree-button"]',
    bouton_affichage = '//*[@id="travel-search-engine-results"]/div[3]/a'
)

In [4]:
cell_elem_decat = scrap_main.CellElem(
    re_desc='heading.*?target="_blank">(.*?)</a>',
    re_diff="Niveau.*value..(.*)..span",
    re_duree="value.*\".(.*)jours.*span",
    re_lieu='destination">(.*?)</div>',
    re_prix="minprice.*strong.(.*)....strong",
    re_theme="theme.*tooltip..(.*?)\""
    )

In [5]:
scrap_main.scraping(pageElem=elemedecat, cellsElem=cell_elem_decat)

Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable


Un fichier en JSON est sauvegardé avec la liste des séjours. 

2. Xpath et RE pour terre d'avenir

In [1]:
import scrap_main

In [2]:
elemeterdav = scrap_main.PageElem(
    nom_site= "Terdav1",
    adresse = "https://www.terdav.com/ps-france/tp-voyage-famille#nos-voyages",
    bouton_accept = '//*[@id="axeptio_btn_acceptAll"]',
    bouton_affichage = '//*[@id="blockResultats"]/a'
)

In [4]:
cell_elem_terdav = scrap_main.CellElem(
    re_desc="name.*?:....(.*?).,",
    re_diff='dimension8.*?:....(.*?).,',
    re_duree="dimension7.*?:.*?'(.*?)Jours",
    re_lieu='dimension6.*?:....(.*?).,',
    re_prix='price.*?:(.*?),',
    re_theme= 'dimension12.*?:....(.*)"'
    )

In [6]:
scrap_main.scraping(pageElem=elemeterdav, cellsElem=cell_elem_terdav)

Element is not clickable
Element is not clickable
Element is not clickable


3. Xpath et RE pour Explora

In [1]:
import scrap_main

In [2]:
elemeexplora = scrap_main.PageElem(
    nom_site= "Explora1",
    adresse = "https://www.explora-project.com/expeditions?",
    bouton_accept = '//*[@id="axeptio_btn_dismiss"]',
    bouton_affichage = '//*[@id="hits"]/button'  
)

In [3]:
cell_elem_explora = scrap_main.CellElem(
    re_desc='"heading-suggestion-padding-white">(.*?)</h3>',
    re_diff="Niveau(.*?)</div>",
    re_duree='calendar.png".*?6px;"/>(.*?)jours',
    re_lieu='hide\-on\-mobile\">(.*?)\-',
    re_prix='jrs.-(.*?)...span',
    re_theme= None
    )

Concernant ce site, nous n'avons pas réussi à automatiser les click d'affichage. Après de multiples tentatives, nous décidons de contourner le problème en cliquant 'à la mainProblème'. Nous ne pouvons donc pas utiliser la fonction scraping.

Voici comment procéder:

In [4]:
elemeexplora.open_site()

La fonction affichage_all_item ne fonctionnant pas, on clique sur le bouton afficher plus en bas de page jusqu'à ce qu'il n'y ait plus d'item à afficher. 

In [5]:
cell_elem_explora.save_sejours_to_Json(elemeexplora)

### Phase de construction et nettoyage du dataframe avec le module makedf.py

In [6]:
import makedf

In [7]:
df = makedf.Df()
df.all_in_one()
df.df

,description,prix,lieux,duree,diff,theme
0,les différents visages du désert,290,dest_proche,4,Découverte,rando nature
1,côte atlantique et essaouira,320,dest_proche,5,Découverte,rando nature
2,l'intégrale des calanques,400,metropole,7,Initié,rando nature
3,le tour de l'île aux fleurs,450,dest_proche,8,Découverte,rando nature
4,"cyclades: serifos, sifnos et milos",580,dest_proche,8,Découverte,rando nature
...,...,...,...,...,...,...
397,de bastia &#224; ajaccio par la c&#244;te ouest,1150,dest_proche,8,Initié,cyclo
398,gr20 nord libert&#233; en h&#244;tel,1330,dest_proche,7,Confirmé,rando nature
399,l&#39;int&#233;gral des chemins de stevenson,1330,metropole,1,Confirmé,rando nature
400,le gr20 int&#233;gral en libert&#233;,1340,dest_proche,1,Confirmé,rando nature


### Phase d'apprentissage et résultats 

Nous n'avons pas encore finaliser l'automatisation de la phase d'apprentissage ds un module dans un module.

In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, ElasticNet, LogisticRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from rich import print
from rich.table import Table

In [13]:
df = pd.read_pickle("my_dataframe.pkl")
X = df.drop(["description", "prix"], axis = 1)
y = df["prix"]
X_tr, X_te, y_tr, y_te = train_test_split(X, y)

In [15]:
one_hot_encoder = Pipeline(
  steps=[
    ('one_hot', OneHotEncoder(handle_unknown='ignore'))
  ]
)

In [16]:
preprocessor = ColumnTransformer(
  transformers=[
    ('categorical', one_hot_encoder, ['lieux', 'diff', 'theme']),
  ]
)

In [17]:
results = dict()

In [19]:
p = Pipeline(
  steps=[
    ('preprocessor', preprocessor),
    ('reg', LinearRegression())
  ]
)

g = GridSearchCV(
    p,
    {
        "reg__fit_intercept": [True, False]
    },
   
)
g.fit(X_tr, y_tr)

results["Lineaire pure"] = g

In [22]:
p = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("Reg",  MLPRegressor())
    ]
)

g = GridSearchCV(
    p,
    {
        'Reg__hidden_layer_sizes': [(50,), (100,), (150,), (200,), (250,), (300,),],
        "Reg__max_iter": [5000],
        "Reg__activation": ["relu", "logistic"],
    },
    n_jobs=-1,
)
g.fit(X_tr, y_tr)
results["reseau neurone"]=g

C:\Users\arkan\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [23]:
p = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("Reg", SVR())
    ]
)

g = GridSearchCV(
    p,
    {
        "Reg__C": [ 0.1, 1., 10., 100,],
        "Reg__epsilon": [0.01, 0.1, 1., 10.]
    },
    n_jobs=-1,
)
g.fit(X_tr, y_tr)
results["SVR"] = g

In [27]:
p = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("Reg", RandomForestRegressor())
    ]
)

g = GridSearchCV(
    p,
    {
        "Reg__n_estimators": range(50, 300, 20),
    },
    n_jobs=-1,
   
)
g.fit(X_tr, y_tr)
results["random forest"] = g

In [28]:
tbl = Table(
    title="Résumé des résultats de crossvalidation.",
    show_header=True,
)
tbl.add_column("Nom")
tbl.add_column("Score Cross validation")
tbl.add_column("Score entrainement")
tbl.add_column("Choix Hyperparamètres")
for nom, modele in results.items():
    tbl.add_row(
        nom, 
        f"{modele.best_score_:.2f}", 
        f"{modele.score(X_tr, y_tr):.2f}",
        str(modele.best_params_),
    )
    
print(tbl)

                          Résumé des résultats de crossvalidation.                           
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Nom            ┃ Score Cross validation ┃ Score entrainement ┃ Choix Hyperparamètres      ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Lineaire pure  │ 0.36                   │ 0.37               │ {'reg__fit_intercept':     │
│                │                        │                    │ True}                      │
│ reseau neurone │ 0.38                   │ 0.47               │ {'Reg__activation':        │
│                │                        │                    │ 'relu',                    │
│                │                        │                    │ 'Reg__hidden_layer_sizes': │
│                │                        │                    │ (300,), 'Reg__max_iter':   │
│                │                        │                    │ 5000}                      │
│ SVR            │ 0.23                   │ 0.30               │ {'Reg__C': 100,            │
│                │                        │                    │ 'Reg__epsilon': 10.0}      │
│ random forest  │ 0.45                   │ 0.49               │ {'Reg__n_estimators': 70}  │
└────────────────┴────────────────────────┴────────────────────┴────────────────────────────┘

Les résultats sont très mauvais, aucn score ne dépasse *O.5*.

Plusieurs pistes : 
- Pas assez de variables explicatives. 
- Echantillon trop petit (388 séjours). Il en faudrait au minimum 1000.
- trop de pertes d'information lors du regroupement des modalités.
- Phase d'optimisation des paramètres très mal configurées.

L'objectif était de prévoir le prix pour certains types de séjours donnés (séjours que nous voulons développer dans le cadre de notre création d'entreprise). Une fois les prix du marchés déterminés, nous aurions fait une phase d'optimisation pour savoir quel est la facturation minimum que nous aurions pu mettre en place en fonction de nos couts et des prix du marché. Nous n'avons pas développé cette phaase car le résultat de la phase d'apprentissage est trop mauvais. 

Il faut d'abord avoir de meilleurs résultats. Nous pourrions peut être abandonné cette troisième phase et nous contenter d'essayer d'anticiper les prix des séjours présentés sur les sites qui n'ont pas encore d'information sur leurs prix.

### Rappel méthode à suivre pour la phase d'apprentissage et de validation

- découpage échantillon en (train/test)
- apprentissage sur échantillon entrainement pour tous les modèles
- comparaison des modèles par cross validation sur echantillon d'entrainement (si pas de CV, découpage échantillon en 3 : train/valid/test)
- predict sur échantillon test (garde fou). 
    - Si surapprentissage (score CV supérieur à score test), reprendre tout à zéro. Pb peut venir d'un découpage malchanceux, d'une CV malchanceuse...
    - Si score test ok, on réentraine sur l'échantillon complet. 

## Quelques remarques 

Nous avons passé beaucoup de temps à assyaer de résoudre ces erreurs sans succès : 

1. Cette erreur est censée être ignorée, mais elle ne l'est pas. Nous avons essayé plusieurs options sans succès

In [6]:
elemeexplora.affichage_all_item()

Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=97.0.4692.71)
Stacktrace:
Backtrace:
	Ordinal0 [0x00646903+2517251]
	Ordinal0 [0x005DF8E1+2095329]
	Ordinal0 [0x004E2848+1058888]
	Ordinal0 [0x004E4F44+1068868]
	Ordinal0 [0x004E4E0E+1068558]
	Ordinal0 [0x004E56BA+1070778]
	Ordinal0 [0x005364F9+1402105]
	Ordinal0 [0x005264D3+1336531]
	Ordinal0 [0x00535BBF+1399743]
	Ordinal0 [0x0052639B+1336219]
	Ordinal0 [0x005027A7+1189799]
	Ordinal0 [0x00503609+1193481]
	GetHandleVerifier [0x007D5904+1577972]
	GetHandleVerifier [0x00880B97+2279047]
	GetHandleVerifier [0x006D6D09+534521]
	GetHandleVerifier [0x006D5DB9+530601]
	Ordinal0 [0x005E4FF9+2117625]
	Ordinal0 [0x005E98A8+2136232]
	Ordinal0 [0x005E99E2+2136546]
	Ordinal0 [0x005F3541+2176321]
	BaseThreadInitThunk [0x75D76739+25]
	RtlGetFullPathName_UEx [0x779B8AFF+1215]
	RtlGetFullPathName_UEx [0x779B8ACD+1165]


2. Pourquoi le module n'arrive pas à transformer les caractères 'parasites' (non décodé) alors que nous y arrivons ci dessous?

In [6]:
import scrap_main
import re

elemeterdav.open_site()
elemeterdav.affichage_all_item()
cellsterdav = elemeterdav.recup_cellule() 
texte = str(cellsterdav[12])
texte = texte.replace("\n", " ")
motif = re.compile("name.*?:....(.*?).,")
b = motif.findall(texte)[0]
b.replace("&#231;", "ç").replace("&#224;", "à").replace("&#39", "'").replace("&#233;", "é")

Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable
Element is not clickable


'cévennes et gorges du tarn en famille'

In [8]:
b.replace("&#231;", "ç").replace("&#224;", "à").replace("&#39", "'").replace("&#233;", "é")

'cévennes et gorges du tarn en famille'

3. Nous n'avons pas eu le temps de décortiquer le code proposé par le professeur lors de la séance 9 qui propse un exemple de module pour la phase d'apprentissage. Nous avons essayé de produire un code qui serait la base du module d'apprentissage (ci-dessous); nous ne comprenons pas pourquoi ce code ne fonctionne pas. 

In [5]:
df = pd.read_pickle("my_dataframe.pkl")
X = df.drop(["description", "prix"], axis = 1)
y = df["prix"]
X_tr, X_te, y_tr, y_te = train_test_split(X, y)


one_hot_encoder = Pipeline(
  steps=[
    ('one_hot', OneHotEncoder(handle_unknown='ignore'))
  ]
)

preprocessor = ColumnTransformer(
  transformers=[
    ('categorical', one_hot_encoder, ['lieux', 'diff', 'theme']),
  ]
)

models_names = ["Linear Regression", "ElasticNet","Random Forest"]

pipelines = [
    Pipeline(steps=[('preprocessor', preprocessor), ('Reg', LinearRegression())]),
    Pipeline(steps=[('preprocessor', preprocessor), ('Reg', ElasticNet())]),
    Pipeline(steps=[('preprocessor', preprocessor), ('Reg', RandomForestRegressor())])
]

allgrid = [
    {"Reg__fit_intercept": [True, False]},
    {
        "Reg__fit_intercept": [True, False],
        "Reg__alpha": [0.1, 1., 10.],
        "Reg__l1_ratio": [0.1, 0.5, 0.9],
        "Reg__max_iter": [10000],
    },
    {"Reg__n_estimators": range(50, 300, 20)}
]

results = dict()
for pipe, grid, name in zip(pipelines, allgrid, models_names):
    g = GridSearchCV(pipe, param_grid= grid)
    g.fit(X_tr, y_tr)
    results[name]=g

In [6]:
tbl = Table(
    title="Résumé des résultats de crossvalidation.",
    show_header=True,
)
tbl.add_column("Nom")
tbl.add_column("Score Cross validation")
tbl.add_column("Score entrainement")
tbl.add_column("Choix Hyperparamètres")
for nom, modele in results.items():
    tbl.add_row(
        nom, 
        f"{modele.best_score_:.2f}", 
        f"{modele.score(X_tr, y_tr):.2f}",
        str(modele.best_params_),
    )
    
print(tbl)

                          Résumé des résultats de crossvalidation.                           
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Nom               ┃ Score Cross validation ┃ Score entrainement ┃ Choix Hyperparamètres   ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Linear Regression │ 0.34                   │ 0.40               │ {'Reg__fit_intercept':  │
│                   │                        │                    │ True}                   │
│ ElasticNet        │ 0.35                   │ 0.40               │ {'Reg__alpha': 0.1,     │
│                   │                        │                    │ 'Reg__fit_intercept':   │
│                   │                        │                    │ True, 'Reg__l1_ratio':  │
│                   │                        │                    │ 0.9, 'Reg__max_iter':   │
│                   │                        │                    │ 10000}                  │
│ Random Forest     │ 0.39                   │ 0.49               │ {'Reg__n_estimators':   │
│                   │                        │                    │ 130}                    │
└───────────────────┴────────────────────────┴────────────────────┴─────────────────────────┘

4. Nous n'avons malheuresement pas eu le temps de réaliser les test. Nous nous demandons cependant quels tests auraie,t été pertinent. 